# Quick Test -- Canyon Treefrog
Runtime during development has been 10-15 seconds.

In [ ]:
# Specify some paths and names in the cell below.
config_path = "C:/"  # Path to folder where you saved your wildlifeconfig file.
filter_set_json = None
taxon_json = None
query_name = 'Quick1'
ask_eBird = False
ask_GBIF = True

In [ ]:
# Nothing to fill out in the next cell
%matplotlib inline
import os
import sqlite3
import pprint
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys
sys.path.append(config_path)
import wranglerconfig as config
sys.path.append(config.codeDir)
import wrangler_functions as functions

# Define some variables
t1 = datetime.now()
working_directory = config.workDir
username = config.gbif_username
password = config.gbif_password
email = config.gbif_email
EBD_file = config.EBD_file
output_database = working_directory + query_name + '.sqlite'

# Set display options
pd.set_option('display.max_colwidth', 45)
pd.set_option('display.max_rows', 150)

print("Notebook run " + str(t1))
print("Results were saved in " + output_database)

## Taxon Concept
Caution! -- Taxon concept mismatches can cause inclusion of innapropriate records or ommission of useful records.

In [ ]:
your_taxon_id = "test1"
gbif_id = 2427572 
ebird_id = None
detection_distance_m = 0
taxon_polygon = "POLYGON ((-114.2805792358642 36.0462394029092, -111.5094157511812 31.24288936279188, -103.0783607856605 28.77403462189241, -102.3729737168321 32.2841750358243, -104.1028515284828 36.2141887050112, -111.9628788668566 37.07073014573142, -114.2805792358642 36.0462394029092))"

In [ ]:
# If a json was provided, use it, otherwise create a new one with info that was provided.
if taxon_json is None:
    # Build a species dictionary
    taxon_info = {"ID": your_taxon_id, "GBIF_ID": gbif_id, "EBIRD_ID": ebird_id, "detection_distance_m": detection_distance_m,
                  "TAXON_EOO": taxon_polygon}

    # Save as json object
    out_file = open(working_directory + your_taxon_id + ".json", "w")  
    json.dump(taxon_info, out_file) 
    out_file.close() 

if taxon_json is not None:
    with open(taxon_json, "r") as f:
        taxon_info = json.load(f)
        f.close()
        
pprint.pprint(taxon_info)

## Filter Set
Specify how you want records filtered and cleaned.  Alternatively, you can load a filter set here by specifying a path in the first cell of this notebook.  

To skip a filter, enter "None" without the quotation marks or "".  

In [ ]:
filter_set_name = "QuickTest1"
print("Filter set name: " + str(filter_set_name))

#### GBIF Request Method

In [ ]:
"""
Whether to get records from GBIF in a darwin core archive.  
"False" uses the GBIF API, which has limitations that may be important.  
"True" requests results be emailed in a darwin core archive.
"""
get_dwca = False 
print("Request a Darwin Core Archive? " + str(get_dwca))

#### Date Limits
Notes: 

In [ ]:
'''
Enter year and month ranges.  For example, years_range = 2015,2017 and months_range = 3,6
'''
years_range = "1995,2020"
months_range = "1,12"
print("Years: " + str(years_range))
print("Months: " + str(months_range))

#### Country
Notes:

In [ ]:
country = "US"
print("Country: " + str(country))

#### Bounding Box
Notes:

In [ ]:
'''
Coordinates should correspond to WGS84 (EPSG:4326).  Don't use this option if you specify a query polygon below.
'''
lat_range = None
lon_range = None
print("Latitude range: " + str(lat_range))
print("Longitude range: " + str(lon_range))

#### Area of Interest
Notes:

In [ ]:
'''
Specify polygons to use for spatial filtering.  
Records with coordinates outside of the polygons will be removed.  
You can specify a geometry for the query and one for the species.  
The species geometry is included to facilitate better handling of taxonomic issues.  
If both are provided, the intersection is calculated and used as the filter.  
The format should be well known text in WGS84 (EPSG 4326), and very importantly, vertices need to be listed counter-clockwise.  
See the ccw_wkt_from_shp() function in wrangler functions for help.
'''
query_polygon = None
print(query_polygon)

#### Taxon EOO
Notes:

In [ ]:
'''
True or False whether you want to apply the taxon EOO to the filtering.  
If True, removes records with centroids outside of the extent of occurrence geometry you provided in taxon_info.
'''
use_taxon_geometry = False
print("Use taxon extent of occurrence? " + str(use_taxon_geometry))

#### Geoissue
Notes:

In [ ]:
'''
Are GBIF records with noted geoissues OK to include? GBIF only.
'''
geoissue = False
print("Records with geoissues OK? " + str(geoissue))

#### Collections
Notes:

In [ ]:
'''
List collection codes that you'd like to omit. GBIF only.
'''
collection_codes_omit = None
print("Omit: " + str(collection_codes_omit))

#### Institutions
Notes:

In [ ]:
'''
List institution codes that you'd like to omit. GBIF only
'''
institutions_omit = ["MCZ"]
print("Omit: " + str(institutions_omit))

#### Datasets
Notes:

In [ ]:
'''
List datasets that you'd like to omit.
'''
datasets_omit = None
print("Omit: " + str(datasets_omit))

#### Coordinate Uncertainty
Notes:

In [ ]:
'''
has_coordinate_uncertainty -- Do you want to remove records without coordinate uncertainty (True) or leave them in the data set
(False)?  Note that eBird records in GBIF (EOD) do not have this and nor do EBD data.  

default_coordUncertainty -- coordinateUncertaintyInMeters is often not provided.  
Here is an option to use a default.  If you don't want anything entered, set this as None.  Note that when the value is
None, georeferences will be approximated with information from other field.  See the User's Guide for more information.

max_coordinate_uncertainty -- A maximum for coordinate uncertainty can also be set in meters.  It should be an integer 
greater than zero.
'''
has_coordinate_uncertainty = False
default_coordUncertainty = 500
max_coordinate_uncertainty = 10000
print("Coordinate uncertainty required? " + str(has_coordinate_uncertainty))
print("Default coordinate uncertainty to use: " + str(default_coordUncertainty))
print("Maximum allowable coordinate uncertainty: " + str(max_coordinate_uncertainty))

#### Bases
Notes:

In [ ]:
'''
List bases of records that you want to omit.  GBIF only.
'''
bases_omit = None
print("Omit: " + str(bases_omit))

#### Sampling Protocols
Notes:

In [ ]:
'''
List sampling protocols that you would like to omit.
'''
sampling_protocols_omit = None
print("Omit: " + str(sampling_protocols_omit))

#### Issues
Notes:

In [ ]:
'''
List issues that you want to omit.  GBIF only.
'''
issues_omit = None
print("Omit: " + str(issues_omit))

#### Duplicates
Notes:

In [ ]:
'''
Specify whether duplicates on latitude, longitude, and date should be included.
'''
duplicates_OK = False
print("Allow duplicates? " + str(duplicates_OK))

#### Filter Set Summary

In [ ]:
if filter_set_json is None:
    # Build a filter set dictionary
    filter_set = {"name": filter_set_name, "query_polygon": query_polygon, "issues_omit": issues_omit,
                  "sampling_protocols_omit": sampling_protocols_omit, "bases_omit": bases_omit,
                  "has_coordinate_uncertainty": has_coordinate_uncertainty, "geoissue": geoissue,
                  "default_coordUncertainty": default_coordUncertainty,
                  "max_coordinate_uncertainty": max_coordinate_uncertainty,
                  "datasets_omit": datasets_omit, "collection_codes_omit": collection_codes_omit,
                  "institutions_omit": institutions_omit, "geoissue": geoissue, "use_taxon_geometry": use_taxon_geometry,
                  "lat_range": lat_range, "lon_range": lon_range, "country": country, 
                  "years_range": years_range, "months_range": months_range, "duplicates_OK": duplicates_OK, "get_dwca": get_dwca}
    
    # Replace empty strings with None
    for x in filter_set.keys():
        if filter_set[x] == "":
            filter_set[x] = None
    
    # Save as json object
    with open(working_directory + filter_set_name + ".json", "w") as f:
        json.dump(filter_set, f) 
        f.close()
        
if filter_set_json is not None:
    with open(filter_set_json, "r") as f:
        filter_set = json.load(f)
        f.close()
        
    # Replace empty strings with None
    for x in filter_set.keys():
        if filter_set[x] == "":
            filter_set[x] = None
        
pprint.pprint(filter_set)

## Processing

In [ ]:
# Create an output database
functions.build_output_database(output_database)

# Save taxon and filter set info into database
output_db_conn= sqlite3.connect(output_database)
cursor = output_db_conn.cursor()
pd.DataFrame(taxon_info.values(), taxon_info.keys()).applymap(str).to_sql(name='taxon_concept', con=output_db_conn, if_exists='replace')
pd.DataFrame(filter_set.values(), filter_set.keys()).applymap(str).to_sql(name='filter_set', con=output_db_conn, if_exists='replace')
output_db_conn.close()

In [ ]:
# Run the appropriate queries
if ask_eBird == True and ask_GBIF == True:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)
    # Run GBIF query
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, query_name, working_directory, username, password, email) 

elif ask_eBird == True and ask_GBIF == False:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)
    gbif_data = None

elif ask_eBird == False and ask_GBIF == True:
    # Run GBIF query
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, query_name, working_directory, username, password, email)
    ebird_data = None

In [ ]:
# Filter out records with undesirable values, locations, and/or duplication.
for x in filter_set.keys():
    if filter_set[x] == "":
        filter_set[x] = None

In [ ]:
functions.process_records(ebird_data=ebird_data, gbif_data=gbif_data, filter_set=filter_set, 
                        taxon_info=taxon_info, working_directory=working_directory, 
                        query_name=query_name)

In [ ]:
output_db_conn= sqlite3.connect(output_database)
cursor = output_db_conn.cursor()
record_count = cursor.execute("SELECT COUNT(record_id) FROM occurrence_records;").fetchone()
print(str(record_count[0]) + " records were saved in the output database")

## Results of the Filtering

#### Attributes Returned for GBIF Records
This count was made before filters were applied

In [ ]:
pd.set_option('display.max_colwidth', 100)
if ask_GBIF == True:
    fields_summary = pd.read_sql("SELECT * FROM gbif_fields_returned", output_db_conn)
    fields_summary.index.name = 'Field'
    pd.set_option('display.max_rows', 250)
    print(fields_summary.sort_values(by="attribute"))
if ask_GBIF == False:
    print("GBIF was not queried.")

#### Attributes Returned for eBird Records
This count was made before filters were applied

In [ ]:
if ask_eBird == True:
    fields_summary = pd.read_sql("SELECT * FROM ebird_fields_returned", output_db_conn)
    fields_summary.index.name = 'Field'
    pd.set_option('display.max_rows', 250)
    print(fields_summary)
if ask_eBird == False:
    print("No eBird Basic Dataset was queried.")

#### Sources

In [ ]:
pd.set_option('display.max_colwidth', 31)
sources = pd.read_sql(sql="SELECT * FROM sources;", con=output_db_conn)
print(sources[['institutionID', 'collectionCode', 'datasetName', 'acquired', 'removed', 'retained']])

#### Bases

In [ ]:
bases = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'basisOfRecord';", con=output_db_conn)
print(bases[['value', 'acquired', 'removed', 'retained']])

#### Protocols

In [ ]:
protocols = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'samplingProtocol';", con=output_db_conn)
if protocols.empty == True:
    print("No protocols were documented.")
if protocols.empty == False:
    print(protocols[['value', 'acquired', 'removed', 'retained']])

#### Issues

In [ ]:
pd.set_option('display.max_colwidth', 90)
issues = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'issues';", con=output_db_conn)
if issues.empty == True:
    print("No issues were documented.")
if issues.empty == False:
    print(issues[['value', 'acquired', 'removed', 'retained']])

#### Establishment Means

In [ ]:
establishment = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'establishmentMeans';", con=output_db_conn)
if establishment.empty == True:
    print("No establishment means were reported.")
if establishment.empty == False:
    print(establishment[['value', 'acquired', 'removed', 'retained']])

#### Identification Qualifiers

In [ ]:
qualifications = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'identificationQualifers';", con=output_db_conn)
if qualifications.empty == True:
    print("No identification qualifiers were reported.")
if qualifications.empty == False:
    print(establishment[['value', 'acquired', 'removed', 'retained']])

## Descriptions of Retained Records

#### Locations

In [ ]:
# Get the record coordinates as a data frame
record_coordinates = (pd.read_sql("""SELECT decimalLatitude, decimalLongitude, radius_m
                                     FROM occurrence_records""",
                                  con=output_db_conn)
                      .astype({'decimalLongitude': 'float', 'decimalLatitude': 'float',
                               'radius_m': 'float'}))

# Make the data frame spatial
gdf = gpd.GeoDataFrame(record_coordinates, geometry=gpd.points_from_xy(record_coordinates['decimalLongitude'],
                                                   record_coordinates['decimalLatitude']))

# Set the coordinate reference system
gdf.crs={'init' :'epsg:4326'}

# Create world map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(figsize=(12,12), color = 'darkkhaki')
gdf.plot(ax=base, marker='o', color='k', markersize=5)
plt.show()

# Create USA map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
if filter_set["country"] == "US":
    usa_bbox = np.array([-124.725839,   24.498131,  -66.949895,   49.384358])
    fig, ax = plt.subplots(figsize=(12,12))
    ax.set_xlim(([usa_bbox[0],  usa_bbox[2]]))
    ax.set_ylim(([usa_bbox[1],  usa_bbox[3]]))
    world.plot(ax=ax, color='darkkhaki')
    gdf.plot(ax=ax, marker='o', color='k', markersize=5)
    plt.show()

# Create coordinate extent map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
os.chdir(config.codeDir)
states = gpd.read_file(os.getcwd() + '/data/us_states.shp')

# Reproject states and record coordinates to facilitate buffering
states = states.to_crs(epsg=5070)
footprints = gdf.to_crs(epsg=5070)

# Buffer points for record footprints
footprints['footprint']=footprints.apply(lambda x: x.geometry.buffer(x.radius_m), axis=1)
footprints.set_geometry(col='footprint', inplace=True, drop=True)

# Map the buffered points/footprints
fig, ax = plt.subplots(figsize=(12,12))
coordinate_bbox = footprints.geometry.total_bounds
ax.set_xlim(([coordinate_bbox[0],  coordinate_bbox[2]]))
ax.set_ylim(([coordinate_bbox[1],  coordinate_bbox[3]]))
states.plot(ax=ax, color = 'darkkhaki')
footprints.boundary.plot(ax=ax, color='k')
plt.show()

# Cleanup
del footprints, gdf

#### Years Represented

In [ ]:
occ_years = [int(x[0]) for x in cursor.execute("SELECT strftime('%Y', eventDate) FROM occurrence_records").fetchall()]
years = filter_set['years_range']
years = years.split(',')
yearsrng = list(range(int(years[0]), int(years[1]), 1))
binsnum = int(years[1]) - int(years[0])
plt.hist(occ_years, bins=binsnum)
plt.ylabel("number of records")
plt.xlabel("year")
plt.xticks(yearsrng, rotation=90)
plt.title("Occurrences per Year")

#### Months Represented

In [ ]:
occ_months = [int(x[0]) for x in cursor.execute("SELECT strftime('%m', eventDate) FROM occurrence_records").fetchall()]
plt.hist(occ_months, bins=range(1, 14), color="g")
plt.ylabel("number of records")
plt.xlabel("month")
plt.xticks(range(1, 13))
plt.title("Occurrences per Month")

#### Distribution of Coordinate Uncertainty Values for Retained Records

In [ ]:
try:
    occ_cert = [float(x[0]) for x in cursor.execute("SELECT coordinateUncertaintyInMeters FROM occurrence_records WHERE coordinateUncertaintyInMeters > 0.0;").fetchall()]
    maxi = max(occ_cert)
    plt.figure(figsize=(16,4))
    plt.hist(occ_cert, bins=50, color="r")
    plt.xticks(range(0, int(maxi), int(maxi/50)), rotation=90)
    plt.ylabel("number of records")
    plt.xlabel("meters")
    plt.title("Reported Coordinate Uncertainties")
except:
    print("No coordinate uncertanties values were reported.")

In [ ]:
try:
    plt.figure(figsize=(11,4))
    plt.boxplot(occ_cert, vert=False)
    plt.xlabel("meters")
    plt.title("Reported Coordinate Uncertainties")
except:
    print("No coordinate uncertanties values were reported.")

In [ ]:
try:
    rng_max = 2000
    occ_cert2 = [x for x in occ_cert if x <= rng_max]
    plt.figure(figsize=(12,4))
    plt.hist(occ_cert2, bins=30, color="m", align='mid')
    plt.xticks(range(0, rng_max + 100, int(rng_max/30.)), rotation=90)
    plt.ylabel("number of records")
    plt.xlabel("meters")
    plt.title("Reported Coordinate Uncertainties Below 2km")
except:
    print("No coordinate uncertanties values were reported.")

#### Distribution of Point-radius Values for Retained Records

In [ ]:
occ_cert = [float(x[0]) for x in cursor.execute("SELECT radius_m FROM occurrence_records;").fetchall()]
plt.figure(figsize=(11,4))
plt.boxplot(occ_cert, vert=False)
plt.xlabel("meters")
plt.title("Compiled Point-radius Buffer Lengths")

#### Distribution of Nominal Coordinate Precisions for Retained Records

In [ ]:
occ_cert = [float(x[0]) for x in cursor.execute("SELECT nominal_xy_precision FROM occurrence_records;").fetchall()]
plt.figure(figsize=(11,4))
plt.boxplot(occ_cert, vert=False)
plt.xlabel("meters")
plt.title("Nominal Precisions of Coordinates")

#### Remarks

In [ ]:
print("General remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT general_remarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Event remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT eventRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Occurrence remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT occurrenceRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Location remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT locationRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Identified remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT general_remarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Georeference remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT georeferenceRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

## Citations
#### eBird 

In [ ]:
if ask_eBird == True:
    with open(config.EBD_file[:-22] + "recommended_citation.txt", "r") as reference:
        print(reference.readlines()[0])
else:
    print("No eBird Basic Dataset was queried")

#### GBIF

In [ ]:
if ask_GBIF == True:
    if filter_set["get_dwca"] == True:
        print("Citations-- ")
        print(cursor.execute("SELECT citations FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to acquire a list of citations")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if filter_set["get_dwca"] == True:
        print("Rights-- ")
        print(output_db_conn.execute("SELECT rights FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to see the rights")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if filter_set["get_dwca"] == True:
        print("DOI-- ")
        doi = output_db_conn.execute("SELECT doi FROM GBIF_download_info").fetchall()[0][0]
        print("https://doi.org/" + doi)
    else:
        print("Set 'get_dwca' to True to perform a search with a doi assigned")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if filter_set["get_dwca"] == True:
        print("GBIF download key-- ")
        print(output_db_conn.execute("SELECT download_key FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to perform a search with a download key assigned")
if ask_GBIF == False:
    print("GBIF was not queried")

## Runtime

In [ ]:
output_db_conn.close()
del cursor
t2 = datetime.now()
print(t2 - t1)

In [ ]:
functions.verify_results(output_database)